In [14]:
sq = scholarly.search_pubs("Machine Learning lncRNA")

In [15]:
p = next(sq)

In [16]:
p

{'container_type': 'Publication',
 'source': <PublicationSource.PUBLICATION_SEARCH_SNIPPET: 'PUBLICATION_SEARCH_SNIPPET'>,
 'bib': {'title': 'Predicting lncRNA-protein interactions by machine learning methods: a review',
  'author': ['ZP Liu'],
  'pub_year': '2020',
  'venue': 'Current Bioinformatics',
  'abstract': 'Here, we aim to provide a review of machine-learning-based methods for predicting lncRNA  of predicting lncRNA-protein interactions into a general framework of machine learning. We'},
 'filled': False,
 'gsrank': 1,
 'pub_url': 'https://www.ingentaconnect.com/content/ben/cbio/2020/00000015/00000008/art00005',
 'author_id': ['zkBXb_kAAAAJ'],
 'url_scholarbib': '/scholar?hl=en&q=info:wfC5OKxQqHMJ:scholar.google.com/&output=cite&scirp=0&hl=en',
 'url_add_sclib': '/citations?hl=en&xsrf=&continue=/scholar%3Fq%3DMachine%2BLearning%2BlncRNA%26hl%3Den%26as_sdt%3D0,33&citilm=1&update_op=library_add&info=wfC5OKxQqHMJ&ei=NyxfZ_frIcyR6rQPhb_isQU&json=',
 'num_citations': 16,
 'citedby

In [17]:
list(p.keys())

['container_type',
 'source',
 'bib',
 'filled',
 'gsrank',
 'pub_url',
 'author_id',
 'url_scholarbib',
 'url_add_sclib',
 'num_citations',
 'citedby_url',
 'url_related_articles']

In [18]:
author_id = 'zkBXb_kAAAAJ'
author = scholarly.fill(scholarly.search_author_id(author_id))

In [19]:
author

{'container_type': 'Author',
 'filled': ['basics',
  'indices',
  'counts',
  'coauthors',
  'publications',
  'public_access'],
 'scholar_id': 'zkBXb_kAAAAJ',
 'source': <AuthorSource.AUTHOR_PROFILE_PAGE: 'AUTHOR_PROFILE_PAGE'>,
 'name': 'Zhi-Ping Liu',
 'affiliation': 'Professor of Biomedical Informatics, Shandong University',
 'organization': 16033752023248436093,
 'interests': ['Bioinformatics',
  'Computational Biology',
  'Systems Biomedicine',
  'Machine Learning',
  'Operations Research'],
 'email_domain': '@sdu.edu.cn',
 'citedby': 5975,
 'citedby5y': 3478,
 'hindex': 37,
 'hindex5y': 28,
 'i10index': 75,
 'i10index5y': 62,
 'cites_per_year': {2011: 34,
  2012: 109,
  2013: 261,
  2014: 328,
  2015: 378,
  2016: 407,
  2017: 458,
  2018: 442,
  2019: 432,
  2020: 445,
  2021: 577,
  2022: 685,
  2023: 621,
  2024: 713},
 'coauthors': [{'container_type': 'Author',
   'filled': [],
   'scholar_id': 'Uoqv8rkAAAAJ',
   'source': <AuthorSource.CO_AUTHORS_LIST: 'CO_AUTHORS_LIST'>,
 

----

In [20]:
p2 = next(sq)

In [21]:
p2

{'container_type': 'Publication',
 'source': <PublicationSource.PUBLICATION_SEARCH_SNIPPET: 'PUBLICATION_SEARCH_SNIPPET'>,
 'bib': {'title': 'Recent advances in predicting protein-lncRNA interactions using machine learning methods',
  'author': ['H Yu', 'ZA Shen', 'YK Zhou', 'PF Du'],
  'pub_year': '2022',
  'venue': 'Current Gene Therapy',
  'abstract': 'classified into the deep learning-based method, the ensemble learning-based method,  and  learning methods. We summarized the state-of-the-art methods in predicting lncRNA-protein'},
 'filled': False,
 'gsrank': 2,
 'pub_url': 'https://www.ingentaconnect.com/content/ben/cgt/2022/00000022/00000003/art00005',
 'author_id': ['', '', '', 'EHvA-IUAAAAJ'],
 'url_scholarbib': '/scholar?hl=en&q=info:WeRErDw2Ap8J:scholar.google.com/&output=cite&scirp=1&hl=en',
 'url_add_sclib': '/citations?hl=en&xsrf=&continue=/scholar%3Fq%3DMachine%2BLearning%2BlncRNA%26hl%3Den%26as_sdt%3D0,33&citilm=1&update_op=library_add&info=WeRErDw2Ap8J&ei=NyxfZ_frIcyR6r

In [23]:
for a in p2['author_id']:
    if a:
        print(a)

EHvA-IUAAAAJ


----

In [2]:
import spacy

# Load the spaCy language model
nlp = spacy.load("en_core_web_sm")

def extract_organizations(affiliation):
    """Extracts organization names from a affiliation value of an author record."""
    organizations = []
    
    doc = nlp(affiliation)
    # Extract entities labeled as ORG (organization)
    for ent in doc.ents:
        if ent.label_ == "ORG":
            organizations.append(ent.text)
    return organizations

In [24]:
import time
from scholarly import scholarly

def search_papers(query, limit=5):
    search_query = scholarly.search_pubs(query)
    papers = []

    for _ in range(limit):
        try:
            paper = next(search_query)
            authors = []
            if "author_id" in paper:
                for author_id in paper["author_id"]:
                    if author_id: # check the author_id is not empty
                        time.sleep(2)  # Introduce delay
                        try:
                            author = scholarly.fill(scholarly.search_author_id(author_id))
                            authors.append({
                                "name": author.get("name"),
                                "affiliation": author.get("affiliation", "Unknown")
                            })
                        except Exception as e:
                            print(f"Error processing author with id {author_id}: {e}")
                        
            papers.append({
                "title": paper.get("bib", {}).get("title"),
                "abstract": paper.get("bib", {}).get("abstract"),
                "authors": authors,
                "year": paper.get("bib", {}).get("pub_year"),
                "url": paper.get("eprint_url", "Unknown")
            })
        except StopIteration:
            break
        #except Exception as e:
        #    print(f"Error processing paper or author: {e}")
        #    continue

    return papers


In [25]:
query = "Machine Learning lncRNA"
papers = search_papers(query, limit=10)

In [13]:
for paper in papers:
    print(f"Title: {paper['title']}")
    print(f"Abstract: {paper['abstract']}")
    print(f"Year: {paper['year']}")
    print(f"URL: {paper['url']}")
    print("Authors:")
    for author in paper["authors"]:
        print(f"  - {author['name']}: {author['affiliation']}")
    print("-" * 80)

Title: Predicting lncRNA-protein interactions by machine learning methods: a review
Abstract: Here, we aim to provide a review of machine-learning-based methods for predicting lncRNA  of predicting lncRNA-protein interactions into a general framework of machine learning. We
Year: 2020
URL: Unknown
Authors:
  - Zhi-Ping Liu: Professor of Biomedical Informatics, Shandong University
--------------------------------------------------------------------------------
Title: Recent advances in predicting protein-lncRNA interactions using machine learning methods
Abstract: classified into the deep learning-based method, the ensemble learning-based method,  and  learning methods. We summarized the state-of-the-art methods in predicting lncRNA-protein
Year: 2022
URL: Unknown
Authors:
  - Pufeng Du: Tianjin University; Tsinghua University; City University of Hong Kong
--------------------------------------------------------------------------------
Title: Recent advances in machine learning methods 

In [28]:
for paper in papers:
    print(f"Title: {paper['title']}")
#    print(f"Abstract: {paper['abstract']}")
#    print(f"Year: {paper['year']}")
#    print(f"URL: {paper['url']}")
    print("Authors:")
    for author in paper["authors"]:
        organizations = extract_organizations(author['affiliation'])
        org = ""
        if len(organizations) > 0:
            org = organizations[0]
            
        print(f"  - {author['name']}: {author['affiliation']}: {organizations}")
        #print(f"  - {author['name']}: {org}")
    print("-" * 80)

Title: Predicting lncRNA-protein interactions by machine learning methods: a review
Authors:
  - Zhi-Ping Liu: Professor of Biomedical Informatics, Shandong University: ['Biomedical Informatics', 'Shandong University']
--------------------------------------------------------------------------------
Title: Recent advances in predicting protein-lncRNA interactions using machine learning methods
Authors:
  - Pufeng Du: Tianjin University; Tsinghua University; City University of Hong Kong: ['Tianjin University', 'Tsinghua University', 'City University of Hong Kong']
--------------------------------------------------------------------------------
Title: Recent advances in machine learning methods for predicting LncRNA and disease associations
Authors:
  - Jianjun Tan: 北京工业大学: []
--------------------------------------------------------------------------------
Title: Prediction of plant lncRNA by ensemble machine learning classifiers
Authors:
  - Caitlin Simopoulos: Roche: ['Roche']
---------

In [29]:
import geopandas as gpd
import folium
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="lncRNA-map")

In [32]:
geolocator.geocode("IUB")

Location(Iub, Dollo, ሶማሌ ክልል / Somali, ኢትዮጵያ, (8.23333, 45.68333, 0.0))

In [32]:
author

{'name': 'Zhi-Ping Liu',
 'affiliation': 'Professor of Biomedical Informatics, Shandong University'}

In [30]:
import geopandas as gpd
import folium
from geopy.geocoders import Nominatim

def geocode_affiliations(papers):
    geolocator = Nominatim(user_agent="lncRNA-map")
    locations = []

    for paper in papers:
        for author in paper["authors"]:
            affiliation = author.get("affiliation")
            if affiliation and affiliation != "Unknown":
                location = geolocator.geocode(affiliation)
                if location:
                    locations.append({
                        "paper_title": paper["title"],
                        "author_name": author["name"],
                        "affiliation": affiliation,
                        "latitude": location.latitude,
                        "longitude": location.longitude
                    })
    return locations

def plot_affiliations(locations):
    # Create a map
    affiliation_map = folium.Map(location=[0, 0], zoom_start=2)
    
    for loc in locations:
        folium.Marker(
            [loc["latitude"], loc["longitude"]],
            popup=f"{loc['author_name']} ({loc['affiliation']})<br>{loc['paper_title']}"
        ).add_to(affiliation_map)
    
    return affiliation_map

# Example usage
locations = geocode_affiliations(papers)
map_object = plot_affiliations(locations)
map_object.save("affiliations_map.html")  # Save to an HTML file


In [33]:
geolocator.geocode("Shandong University")

Location(山东大学（青岛校区）, 72, 滨海公路, 青岛蓝谷高新技术产业开发区, 即墨区, 青岛市, 山东省, 266200, 中国, (36.36553935, 120.68458591160784, 0.0))

In [34]:
geolocator.geocode("Professor of Biomedical Informatics, Shandong University")

TODO:

Need to extract the named entity of organization/university from the affilication field

In [9]:
import arxiv

def search_papers_arxiv(query, limit=20):
    """
    Search for papers on arXiv based on a query using the updated Client.results method.
    
    Args:
        query (str): Search query for arXiv (e.g., "Machine Learning lncRNA").
        limit (int): Maximum number of results to retrieve.
    
    Returns:
        List[Dict]: A list of dictionaries containing paper metadata.
    """
    client = arxiv.Client(
        page_size=limit  # Defines the number of results per API call
    )
    search = arxiv.Search(
        query=query,
        max_results=limit,
        sort_by=arxiv.SortCriterion.Relevance
    )
    
    papers = []
    for result in client.results(search):
        papers.append({
            "title": result.title,
            "abstract": result.summary,
            "institution": result.authors[0].affiliation if result.authors and result.authors[0].affiliation else "Unknown",
            "year": result.published.year,
            "url": result.entry_id
        })
    
    return papers



In [3]:
import spacy

nlp = spacy.load("en_core_web_sm")
model_keywords = ["SVM", "Random Forest", "Neural Network", "Deep Learning", "Gradient Boosting"]

def categorize_papers(papers):
    for paper in papers:
        doc = nlp(paper["abstract"])
        paper["ml_model"] = [
            keyword for keyword in model_keywords if keyword in doc.text
        ]
    return papers


In [4]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="geoapi")

def geocode_institution(papers):
    for paper in papers:
        location = geolocator.geocode(paper["institution"])
        if location:
            paper["latitude"] = location.latitude
            paper["longitude"] = location.longitude
    return papers


In [5]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

def create_geodataframe(papers):
    df = pd.DataFrame(papers)
    geometry = [Point(xy) for xy in zip(df["longitude"], df["latitude"])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry)
    return gdf


In [6]:
import geopandas as gpd
import matplotlib.pyplot as plt

def plot_papers_on_map(gdf):
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    ax = world.plot(color="white", edgecolor="black")
    gdf.plot(ax=ax, color="red", markersize=5)
    plt.show()


In [11]:
query = "Machine Learning lncRNA"

In [12]:
papers = search_papers(query, limit=50)

In [26]:
papers = categorize_papers(papers)

In [10]:
query = "Machine Learning lncRNA"
papers = search_papers_arxiv(query, limit=10)
for paper in papers:
    print(f"Title: {paper['title']}")
    print(f"Abstract: {paper['abstract']}")
    print(f"Institution: {paper['institution']}")
    print(f"Year: {paper['year']}")
    print(f"URL: {paper['url']}")
    print("-" * 80)



AttributeError: 'Author' object has no attribute 'affiliation'